In [1]:
pip install google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install google-auth-oauthlib google-auth google-api-python-client qrcode[pil] Pillow


Note: you may need to restart the kernel to use updated packages.


In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox, font  # Import font module
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import qrcode
from PIL import Image, ImageTk
import os

# Function to authenticate and create the Google Drive API service
def authenticate_drive():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                r"D:\Python\credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('drive', 'v3', credentials=creds)

# Function to upload a PDF file to Google Drive
def upload_pdf_to_drive():
    global drive_service
    file_path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
    if file_path:
        file_name = os.path.basename(file_path)
        file_metadata = {'name': file_name, 'mimeType': 'application/pdf'}
        media = MediaFileUpload(file_path, mimetype='application/pdf')
        uploaded_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        file_id = uploaded_file.get('id')
        file_link = make_file_public(file_id)
        entry_link.delete(0, tk.END)
        entry_link.insert(0, file_link)
        generate_qr_code(file_link)
    else:
        messagebox.showerror("Error", "No file selected!")

# Function to upload an image file to Google Drive
def upload_image_to_drive():
    global drive_service
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.gif")])
    if file_path:
        file_name = os.path.basename(file_path)
        file_metadata = {'name': file_name, 'mimeType': 'image/jpeg'}
        media = MediaFileUpload(file_path, mimetype='image/jpeg')
        uploaded_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        file_id = uploaded_file.get('id')
        file_link = make_file_public(file_id)
        entry_link.delete(0, tk.END)
        entry_link.insert(0, file_link)
        generate_qr_code(file_link)
    else:
        messagebox.showerror("Error", "No file selected!")

# Function to make the uploaded file public and get the shareable link
def make_file_public(file_id):
    permission = {'type': 'anyone', 'role': 'reader'}
    drive_service.permissions().create(fileId=file_id, body=permission).execute()
    file = drive_service.files().get(fileId=file_id, fields='webViewLink').execute()
    return file['webViewLink']

# Function to generate a QR code from the input text
def generate_qr_code(text):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(text)
    qr.make(fit=True)
    img = qr.make_image(fill='black', back_color='white')
    img_path = os.path.join(os.getcwd(), 'qr_code.png')
    img.save(img_path)
    load_qr_image(img_path)

# Function to load and display the QR code image
def load_qr_image(img_path):
    img = Image.open(img_path)
    img.thumbnail((200, 200))
    img = ImageTk.PhotoImage(img)
    qr_label.config(image=img)
    qr_label.image = img

# Functions to handle the button click events for different QR code types
def generate_url_qr():
    url = entry_url.get()
    if url:
        generate_qr_code(url)
    else:
        messagebox.showerror("Error", "URL field is empty!")

def generate_email_qr():
    email = entry_email.get()
    if email:
        generate_qr_code(f"mailto:{email}")
    else:
        messagebox.showerror("Error", "Email field is empty!")

def generate_phone_qr():
    phone = entry_phone.get()
    if phone:
        generate_qr_code(f"tel:{phone}")
    else:
        messagebox.showerror("Error", "Phone number field is empty!")

# Initialize the main application window
root = tk.Tk()
root.title("PDF/Image to QR Code")

# Set the background color of the main window to purple
root.configure(bg='purple')

# Drive API authentication
SCOPES = ['https://www.googleapis.com/auth/drive.file']
drive_service = authenticate_drive()

# Create a large font for the title
title_font = font.Font(family='Helvetica', size=48, weight='bold')

# Create and place the title label
title_label = tk.Label(root, text="QR CODE GENERATOR", font=title_font, bg='purple', fg='white')
title_label.pack(pady=20)

# Create and place the UI elements for PDF upload
btn_upload_pdf = tk.Button(root, text="Upload PDF to Google Drive", command=upload_pdf_to_drive, bg='yellow', fg='black')
btn_upload_pdf.pack(pady=10)

# Create and place the UI elements for image upload
btn_upload_image = tk.Button(root, text="Upload Image to Google Drive", command=upload_image_to_drive, bg='orange', fg='black')
btn_upload_image.pack(pady=10)

# Create and place the UI elements for shareable link
lbl_link = tk.Label(root, text="Shareable Link:", bg='red', fg='white')
lbl_link.pack()

entry_link = tk.Entry(root, width=50)
entry_link.pack(pady=5)

btn_generate_link_qr = tk.Button(root, text="Generate QR Code for Link", command=lambda: generate_qr_code(entry_link.get()), bg='blue', fg='white')
btn_generate_link_qr.pack(pady=5)

# Create and place the UI elements for URL QR code
lbl_url = tk.Label(root, text="Enter URL:", bg='green', fg='white')
lbl_url.pack()

entry_url = tk.Entry(root, width=50)
entry_url.pack(pady=5)

btn_url = tk.Button(root, text="Generate QR Code for URL", command=generate_url_qr, bg='blue', fg='white')
btn_url.pack(pady=5)

# Create and place the UI elements for Email QR code
lbl_email = tk.Label(root, text="Enter Email:", bg='green', fg='white')
lbl_email.pack()

entry_email = tk.Entry(root, width=50)
entry_email.pack(pady=5)

btn_email = tk.Button(root, text="Generate QR Code for Email", command=generate_email_qr, bg='blue', fg='white')
btn_email.pack(pady=5)

# Create and place the UI elements for Phone QR code
lbl_phone = tk.Label(root, text="Enter Phone Number:", bg='green', fg='white')
lbl_phone.pack()

entry_phone = tk.Entry(root, width=50)
entry_phone.pack(pady=5)

btn_phone = tk.Button(root, text="Generate QR Code for Phone", command=generate_phone_qr, bg='blue', fg='white')
btn_phone.pack(pady=5)

# Label to display the QR code image
qr_label = tk.Label(root)
qr_label.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()
